# 인공지능으로 세상에 없던 새로운 패션 만들기
-----
### 프로젝트 - CIFAR-10 이미지 생성하기

## 실습목표

>* 생성 모델링 개념을 이해하며 판별 모델링과의 차이 알기
>* Pix2Pix, CycleGAN 등의 이미지 관련 다양한 생성 모델링의 응용을 접하며 흥미 가지기
>* Fashion MNIST 데이터셋의 의미를 알기
>* 생성적 적대 신경망(GAN)의 구조와 원리를 이해하기
>* 텐서플로우로 짠 DCGAN 학습 코드를 익히며 응용하기

## 순서
> **Step1.** 작업환경 구성하기<br>
**Step2.** 데이터셋 구성하기<br>
**Step3.** 생성자 모델 구현하기<br>
**Step4.** 판별자 모델 구현하기<br>
**Step5.** 손실함수와 최적화 함수 구현하기<br>
**Step6.** 훈련과정 상세 기능 구현하기<br>
**Step7.** 학습 과정 진행하기<br>
**Step8.** (optional) GAN 훈련 과정 개선하기

## 용어정리
---

**✓ GAN(Generative Adversarial Network)**이란?
>* 비지도 학습에 사용되는 인공지능 알고리즘
>* 제로섬 게임 틀 안에서 서로 경쟁하는 두 개의 신경 네트워크 시스템에 의해 구현(2014년에 이안 굿펠로우)

>* **확률분포(Probability Distribution)**란?
>> * 확률 변수가 특정한 값을 가질 확률을 나타내는 함수를 의미
>> * Generator가 들어가는 모든 모델에 대해 해당하는 내용

>* **랜덤변수(Random Variable)**이란?
>> * 측정할 때마다 다른 값
>> * 특정한 확률분포를 따르는 숫자를 생성
>> * 랜덤변수에 대한 확률분포를 안다는 이야기는 랜덤변수 즉 데이터에 대한 전부를 이해하고 있다는 것과 같음!

GAN을 학습하는 과정은 보기보다 까다롭습니다. 모델 구조, learning_rate, 파라미터 초기화, 생성자와 판별자의 경쟁 관계, 데이터의 특성 등 고려해야 할 것이 참 많습니다.

이번에는 앞에서 배운 FASHION-MNIST 데이터 생성용 DCGAN 모델구조를 이용해서 CIFAR-10 데이터를 생성하는 모델을 직접 만들어 봅시다.
모델 구현 및 학습의 전과정의 흐름은 거의 비슷하겠지만, 아래와 같이 몇 가지 달라지는 점이 있습니다.

* 이미지 데이터의 shape가 (28, 28, 1)에서 (32, 32, 3)으로 변경됩니다. 생성자, 판별자 모델의 입출력 shape 및 모델 구조에 영향이 있습니다.
* 이미지가 단색의 grayscale에서 RGB 3채널의 컬러이미지로 변경됩니다. 시각화 과정에서 고려할 점이 있습니다.
* 입력데이터 전체 차원이 3~4배 증가하면서, 학습이 진행되는 양상이 다소 달라집니다.

### Step1. 작업환경 구성하기
---
**✓ 다음과 같이 작업환경을 구성합시다.**

> $ mkdir -p ~/aiffel/dcgan_newimage/cifar10/generated_samples

> $ mkdir -p ~/aiffel/dcgan_newimage/cifar10/training_checkpoints

> $ mkdir -p ~/aiffel/dcgan_newimage/cifar10/training_history

BeautifulSoup 라이브러리도 설치하지 않았다면 설치해주세요.

> $ pip install beautifulsoup4

nltk 패키지에서 불용어 사전을 다운로드 받고, 데이터 전처리를 위한 나머지 패키지도 함께 불러옵니다.

In [5]:
import nltk  # 영어 기호, 통계, 자연어 처리를 위한 라이브러리
nltk.download('stopwords')  # NLTK의 불용어(stopwords)를 사용
from nltk.corpus import stopwords

import numpy as np  # C언어로 구현된 파이썬 라이브러리, pandas와 matplotlib의 기반으로 사용
import pandas as pd  #  데이타 분석(Data Analysis)을 위해 사용(1차원 자료구조인 Series, 2차원 자료구조인 DataFrame, 그리고 3차원 자료구조인 Panel을 지원)
import os  # 운영 체제와 상호 작용하기 위한 수십 가지 함수들을 제공
import re  # 정규 표현식을 사용하고자 할 때 사용(regex의 약자)
import matplotlib.pyplot as plt  # 라인 플롯, 바 차트, 파이차트, 히스토그램, Box Plot, Scatter Plot 등을 비롯하여 다양한 차트와 플롯 스타일을 지원
from bs4 import BeautifulSoup  # 문서를 파싱하는데 사용하는 패키지
from tensorflow.keras.preprocessing.text import Tokenizer  # 케라스(Keras)의 토큰화 함수를 이용해 텍스트 전처리
from tensorflow.keras.preprocessing.sequence import pad_sequences  # padding을 사용하여 길이를 일정하게 만들며 sequences에는 길이가 다른 데이터가 들어감
import urllib.request  # URL 열기 및 읽기

[nltk_data] Downloading package stopwords to /home/aiffel-
[nltk_data]     dj49/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# import urllib.request

urllib.request.urlretrieve("https://raw.githubusercontent.com/sunnysai12345/News_Summary/master/news_summary_more.csv", filename="news_summary_more.csv")
data = pd.read_csv('news_summary_more.csv', encoding='iso-8859-1')

data.sample(10)

,headlines,text
4197,Panel to conduct survey for sub-categorisation...,A commission examining the sub-categorisation ...
92225,Minister Babul Supriyo shares wrong pic of Raj...,Union Minister Babul Supriyo has shared a phot...
69216,Domestic worker arrested for burglary at Myntr...,The Bengaluru Police has arrested Myntra CEO A...
6429,B'luru home services platform Housejoy raises ...,Bengaluru-based online home services platform ...
56295,Darjeeling toy train resumes operations after ...,"The Darjeeling toy train, also known as the Da..."
5739,Saturn losing its rings at 'worst-case-scenari...,A new NASA research has confirmed Saturn is lo...
19544,Ex-Andhra CM's son could've survived had he wo...,"Former Andhra Pradesh CM NTR Rao's son, Nandam..."
48403,"What is Vantablack, used to make 'world's dark...","Dubbed the ""world's blackest black"", Vantablac..."
10878,11 properties of Nirav Modi worth Ã¢ÂÂ¹56 cr ...,The Enforcement Directorate (ED) on Tuesday sa...
74889,"I-T Dept summons Lalu's daughter Misa Bharti, ...",The Income Tax Department on Thursday asked RJ...


이 데이터는 기사의 본문에 해당되는 text와 headlines 두 가지 열로 구성되어져 있습니다.

추상적 요약을 하는 경우에는 text를 본문, headlines를 이미 요약된 데이터로 삼아서 모델을 학습할 수 있어요. 추출적 요약을 하는 경우에는 오직 text열만을 사용하세요.

In [8]:
data = data[['text','headlines']]
data.head()

#랜덤한 5개 샘플 출력
data.sample(5)

,text,headlines
59639,Indian pacer Bhuvneshwar Kumar tied the knot w...,Bhuvneshwar Kumar ties the knot with childhood...
81604,US First Lady Melania Trump was sent in to end...,Melania tried to end 'overtime' meeting betwee...
23147,"Marie Laguerre, who was attacked by her harass...",French woman hit by harasser starts online for...
53552,Pakistan's PM Shahid Abbasi has claimed his co...,We received very insignificant amount of US ai...
3387,"Ayushmann Khurrana's wife Tahira Kashyap, who ...",Can't keep calm: Ayushmann's wife Tahira on co...


**Error]** **<span style="color:Orange">에러명</span>**: 사유

### <span style="color:green">Step2. 데이터 전처리하기 (추상적 요약)</span>
---
이제 데이터를 불러왔으니 전처리를 진행해 볼게요. 빈칸으로 존재하는 null 데이터, 의미는 같지만 다른 식으로 작성된 글 같은 중복 항목과 같은 학습할 때 방해가 되는 데이터를 먼저 솎아냅시다. 만약, 불용어 제거를 선택한다면 상대적으로 길이가 짧은 요약 데이터에 대해서도 불용어를 제거하는 것이 좋을지 고민해보세요.

**✓ 중복 샘플과 NULL 값이 존재하는 샘플 제거 : 데이터의 중복 샘플 유무를 확인**

In [11]:
print('text 열에서 중복을 배제한 유일한 샘플의 수 :', data['text'].nunique(),'개')
print('headlines 열에서 중복을 배제한 유일한 샘플의 수 :', data['headlines'].nunique(),'개')

text 열에서 중복을 배제한 유일한 샘플의 수 : 98360 개
headlines 열에서 중복을 배제한 유일한 샘플의 수 : 98280 개


In [16]:
# 데이터프레임의 drop_duplicates()를 사용하면, 손쉽게 중복 샘플을 제거
data.drop_duplicates(subset = ['text'], inplace = True)

print('전체 샘플수 :',(len(data)),'개')

전체 샘플수 : 98360 개


In [13]:
print(data.isnull().sum())  # 이터프레임에 Null 값이 있는지 확인하는 방법

text         0
headlines    0
dtype: int64


In [18]:
# 데이터프레임에서 Null을 제거할 때는 dropna() 함수를 사용
data.dropna(axis = 0, inplace = True)
print('전체 샘플수 :',(len(data)),'개')

전체 샘플수 : 98360 개


**✓ 텍스트 정규화와 불용어 제거**

In [ ]:
contractions = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",
                           "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",
                           "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",
                           "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",
                           "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",
                           "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",
                           "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",
                           "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",
                           "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",
                           "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",
                           "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",
                           "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",
                           "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",
                           "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",
                           "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",
                           "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",
                           "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",
                           "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",
                           "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",
                           "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",
                           "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
                           "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",
                           "you're": "you are", "you've": "you have"}

print("정규화 사전의 수: ",len(contractions),'개')

In [ ]:
print('불용어 개수 :', len(stopwords.words('english') ))
print(stopwords.words('english'))

### <span style="color:green">Step3. 어텐션 메커니즘 사용하기 (추상적 요약)</span>
---
일반적인 seq2seq보다는 어텐션 메커니즘을 사용한 seq2seq를 사용하는 것이 더 나은 성능을 얻을 수 있어요. 실습 내용을 참고하여 어텐션 메커니즘을 사용한 seq2seq를 설계해 보세요.

### <span style="color:green">Step4. 실제 결과와 요약문 비교하기 (추상적 요약)</span>
---
실습에서 사용된 전처리를 참고하여 각자 필요하다고 생각하는 전처리를 추가 사용하여 텍스트를 정규화 또는 정제해 보세요. 만약, 불용어 제거를 선택한다면 상대적으로 길이가 짧은 요약 데이터에 대해서도 불용어를 제거하는 것이 좋을지 고민해보세요.

### <span style="color:green">Step5. Summa을 이용해서 추출적 요약해보기</span>
---
추상적 요약은 추출적 요약과는 달리 문장의 표현력을 다양하게 가져갈 수 있지만, 추출적 요약에 비해서 난이도가 높아요. 반대로 말하면 추출적 요약은 추상적 요약에 비해 난이도가 낮고 기존 문장에서 문장을 꺼내오는 것이므로 잘못된 요약이 나올 가능성이 낮아요.

Summa의 summarize를 사용하여 추출적 요약을 해보세요.

# <span style="color:purple">루브릭 평가 기준</span>
---
번호|평가문항|상세기준
---|---|---
1 |Abstractive 모델 구성을 위한 텍스트 전처리 단계가 체계적으로 진행되었다.|분석단계, 정제단계, 정규화와 불용어 제거, 데이터셋 분리, 인코딩 과정이 빠짐없이 체계적으로 진행되었다.
2 |텍스트 요약모델이 성공적으로 학습되었음을 확인하였다.|모델학습이 안정적으로 수렴되었음을 그래프를 통해 확인하였으며, 실제 요약문과 유사한 요약문장을 얻을 수 있었다.
3 |Extractive 요약을 시도해 보고 Abstractive 요약 결과과 함께 비교해 보았다.|두 요약 결과를 문법완성도 측면과 핵심단어 포함 측면으로 나누어 비교분석 결과를 제시하였다.

# <span style="color:purple">회고</span>
---